In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
# Import the necessary libraries to mount Google Drive
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Define the path to your AID dataset
train_dir = '/content/drive/MyDrive/Dataset/AID'
valid_dir = '/content/drive/MyDrive/Dataset/AIDValid'

# Specify layer names for feature extraction
#layer_names = ['block1_pool', 'block2_pool', 'block3_pool', 'block4_pool']

# Load the VGG19 model
model = VGG19(weights='imagenet', include_top=True)

# Specify the layers for concatenation
layer_name1 = 'block1_pool'
layer_name2 = 'block2_pool'
layer_name3 = 'block3_pool'
layer_name4 = 'block4_pool'

# Extract the outputs of the desired layers
output1 = model.get_layer(layer_name1).output
x= MaxPooling2D(pool_size=(2, 2))(output1)
x= MaxPooling2D(pool_size=(2, 2))(x)
x= MaxPooling2D(pool_size=(2, 2))(x)
output2 = model.get_layer(layer_name2).output
x1= MaxPooling2D(pool_size=(2, 2))(output2)
x1= MaxPooling2D(pool_size=(2, 2))(x1)
x1=Conv2D(64,(1,1), activation='relu')(x1)
output3 = model.get_layer(layer_name3).output
x2= MaxPooling2D(pool_size=(2, 2))(output3)
x2=Conv2D(128,(1,1), activation='relu')(x2)
x2=Conv2D(64,(1,1), activation='relu')(x2)
output4 = model.get_layer(layer_name4).output
x3=Conv2D(256,(1,1), activation='relu')(output4)
x3=Conv2D(128,(1,1), activation='relu')(x3)
x3=Conv2D(64,(1,1), activation='relu')(x3)

# Concatenate the activations of the four layers along the channel axis (axis=-1)
concatenated_activations = tf.concat([x, x1,x2,x3], axis=-1)

x = GlobalAveragePooling2D()(concatenated_activations)

x = Dense(256, activation='relu')(x)
output = Dense(30, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

# Data generators
train_generator = ImageDataGenerator(...).flow_from_directory(train_dir)
valid_generator = ImageDataGenerator(...).flow_from_directory(valid_dir)

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Train model
model.fit(train_generator,validation_data=valid_generator,epochs=30)

# Save the trained model
model.save('vgg19_aid_final.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Import the necessary libraries to mount Google Drive
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Load the trained model
loaded_model = load_model('vgg19_aid_final.h5')
# Access class names from the training generator
#class_names = train_generator.class_indices
#print(class_names)
# Load and preprocess an image for prediction
img_path = '/content/drive/MyDrive/airport.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize the image

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the class with the highest probability
predicted_class = np.argmax(predictions)

# Print the predicted class
print(f"The image belongs to class {predicted_class}")